# Siamese Model

## Imports

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torchvision.datasets import Omniglot
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.optim as optim
from tqdm.notebook import trange, tqdm
from PIL import Image
import random
from dataset import OmniglotDataset, kWay_nShotDataset
from train_funcs import trainSiamese,testSiamese_kway_nshot
from saving import save_checkpoint, load_checkpoint
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import onnx
from onnxruntime.quantization import quantize_static, QuantType

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

## Data Loaders

In [ ]:
print(f'Downloading Omniglot dataset')
root = "/Data"
Omniglot(root=root, background=True, download=True, transform=transforms.ToTensor())
Omniglot(root=root, background=False, download=True, transform=transforms.ToTensor())
    

In [ ]:
train_root = "/data/omniglot-py/images_background"
test_root = "/data/omniglot-py/images_evaluation"

data_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=15,translate=(0.1,0.1),scale=(0.9, 1.1), fill=255),
    transforms.ToTensor()
])

train_dataset = OmniglotDataset(train_root,30000*8,transform=data_transforms)
val_dataset = OmniglotDataset(train_root,10000,transform=data_transforms)
test_dataset = kWay_nShotDataset(test_root,500,kway=40,nshot=5,transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

## Architecture

In [ ]:
        
class SiameseModel(nn.Module):
    def __init__(self, ):
        super(SiameseModel,self).__init__()
        self.conv = nn.Sequential(
            #1 @ 105x105
            nn.Conv2d(in_channels=1,out_channels=64,kernel_size=10),
            nn.ReLU(),
            #64 @ 96x96
            nn.MaxPool2d(kernel_size=(2,2)),
            
            #64 @ 48x48
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=7),
            nn.ReLU(),
            #128 @ 42x42
            nn.MaxPool2d(kernel_size=(2,2)),
            
            #128 @ 21x21
            nn.Conv2d(in_channels=128,out_channels=128,kernel_size=4),
            nn.ReLU(),
            #128 @ 18x18
            nn.MaxPool2d(kernel_size=(2,2)),
            
            #128 @ 9x9
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4),
            nn.ReLU(),
            #256 @ 6x6
        )
        
        # HCC-129: Add dropout layer to model
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.sig = nn.Sigmoid()
        self.fc2 = nn.Linear(4096, 1)
        
    def calculateEmbedding(self,x):
        x = self.conv(x)
        x = x.view(-1,256 * 6 * 6)
        x = self.sig(self.fc1(x))
        return x
    
    def forward(self,x1,x2):
        x = torch.abs(x1 - x2)
        x = self.fc2(x)
        
        return x
        


In [ ]:
model = SiameseModel()

print(f"Model Architecture: {model.cuda()}\n")
print(f"Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


## Training

In [ ]:
#Hyperparameters
EPOCHS = 500
learning_rate = 0.00008

In [ ]:
model.cuda()

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCEWithLogitsLoss()

train_loss_history = []
val_loss_history = []
total_training_time = 0.0
print_rate=10


In [ ]:

print("Training Started\n")
for i in range(EPOCHS):
    
    now = time.time()
    
    train_loss, val_loss = trainSiamese(
                                        model=model,
                                        train_dataloader=train_loader,
                                        val_dataloader=val_loader,
                                        optimizer=optimizer,
                                        criterion=criterion)
    
    end = time.time()
    
    epoch_time = (end - now) / 60
    
    total_training_time += epoch_time 
    
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)
    print(f"Epoch {i+1}/{EPOCHS}")
    if i % print_rate == 0:
        print(f"Epoch {i+1}/{EPOCHS}, Train_Loss: {train_loss:.4f}, Val_Loss: {val_loss:.4f}")
        print(f"Time for Epoch({i+1}): {epoch_time:.2f} Minutes\n")
        save_checkpoint(i,model=model,optimizer=optimizer,train_loss_history=train_loss_history,val_loss_history=val_loss_history)
    
print(f"Total training time: {total_training_time} Minutes")
    


In [ ]:
#Load model
model = SiameseModel()
model.cuda()
train_loss_history, val_loss_history = load_checkpoint("500_checkpoint_08_11.pt",model=model,optimizer=optimizer)

## Plotting Loss

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_loss_history, label='Train Loss', color='blue', marker='o')
plt.plot(val_loss_history, label='Validation Loss', color='darkorange', marker='o')

# Adding labels, legend, and grid
plt.title('Train and Validation Loss History', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot Train Loss
axes[0].plot(train_loss_history, label='Train Loss', color='blue', marker='o')
axes[0].set_title('Training Loss History', fontsize=16)
axes[0].set_xlabel('Epochs', fontsize=14)
axes[0].set_ylabel('Loss', fontsize=14)
axes[0].legend(fontsize=12)
axes[0].grid(True)

# Plot Validation Loss
axes[1].plot(val_loss_history, label='Validation Loss', color='darkorange', marker='o')
axes[1].set_title('Validation Loss History', fontsize=16)
axes[1].set_xlabel('Epochs', fontsize=14)
axes[1].set_ylabel('Loss', fontsize=14)
axes[1].legend(fontsize=12)
axes[1].grid(True)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


## Testing

The model will be tested over 500 samples in 40-way 5-shot recognition

In [ ]:
accuracy = testSiamese_kway_nshot(model,test_loader)

## Exporting to ONNX

Load The Weights

In [ ]:
model = SiameseModel()
model.cuda()
load_checkpoint("500_checkpoint_08_11.pt",model=model,optimizer=optimizer)

Wrap the embedding model in a wrapper

In [ ]:
class EmbeddingWrapper(nn.Module):
    def __init__(self, model):
        super(EmbeddingWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model.calculateEmbedding(x)


embedding_model = EmbeddingWrapper(model)
embedding_model = embedding_model.cuda()


In [ ]:
dummy_input_image = torch.randn(1, 1, 105, 105).cuda() 

torch.onnx.export(
    embedding_model,                
    dummy_input_image,                        
    "siamese_embedding_model_500.onnx",                   
    input_names=["input_image"],              
    output_names=["embedding"],               
    dynamic_axes={"input_image": {0: "batch_size"}, "embedding": {0: "batch_size"}},  
    opset_version=11
)



Export Comparison Model (Not used in final app)

In [ ]:
dummy_embedding1 = torch.randn(1, 4096).cuda()  
dummy_embedding2 = torch.randn(1, 4096).cuda()  

torch.onnx.export(
    model,                                
    (dummy_embedding1, dummy_embedding2),  
    "siamese_comparison_model_500.onnx",       
    input_names=["embedding1", "embedding2"],  
    output_names=["output"],               
    dynamic_axes={
        "embedding1": {0: "batch_size"},   
        "embedding2": {0: "batch_size"},   
        "output": {0: "batch_size"}
    },
    opset_version=11
)


# Quantization

Quantize Embedding Model (Not used in final App)

In [ ]:
class SiameseCalibrationDataReader(CalibrationDataReader):
    def __init__(self, dataloader, num_batches):

        self.dataloader = iter(dataloader)
        self.num_batches = num_batches
        self.batch_count = 0
        self.data = None

    def get_next(self):
        if self.batch_count < self.num_batches:
            try:
                img1, _ , _ = next(self.dataloader)
                
                img1_np = img1.numpy() 
                
                self.data = {"input_image": img1_np}
                self.batch_count += 1
                return self.data
            except StopIteration:
                return None
        else:
            return None

    def rewind(self):
        self.dataloader = iter(self.dataloader)
        self.batch_count = 0


In [ ]:


calibration_data_reader = SiameseCalibrationDataReader(dataloader=train_loader, num_batches=10)

onnx_model_path = "siamese_embedding_model_500.onnx"
quantized_model_path = "siamese_embedding_model_500_quantized.onnx"

quantize_static(
    model_input=onnx_model_path,
    model_output=quantized_model_path,
    calibration_data_reader=calibration_data_reader,
    weight_type=QuantType.QInt8
)